In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def prepare_data(dataset,Useless_fatures, name_target = 'target', replace_Nan = 0):
    dataset['ps_car_13_x_ps_reg_03'] = dataset['ps_car_13'] * dataset['ps_reg_03']
    #transform float 64 -> 32
    for c, dtype in zip(dataset.columns, dataset.dtypes): 
        if dtype == np.float64:     
            dataset[c] = dataset[c].astype(np.float32)
            
    if replace_Nan != False:
        dataset.replace(-1, replace_Nan )

    if name_target in dataset:
        X = dataset.drop([name_target],1)
        X = X.drop(Useless_fatures,1)
        
        label = dataset[name_target]
        y = label.as_matrix()
        print(len(X), " ",len(label))
        return X , y
    else:
        return dataset.drop(Useless_fatures,1)
    

In [3]:
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
target_0 = train['target']==0
target_1 = train['target']==1
train_down_samp = train[target_1].append(train[target_0][0:int(sum(target_1)*4)])
train_down_samp = train_down_samp.append(train[target_1])
train_down_samp = train_down_samp.append(train[target_1])
train_down_samp = train_down_samp.append(train[target_1])
to_drop = ['ps_car_11_cat', 'ps_ind_14', 'ps_car_11', 'ps_car_14', 'ps_ind_06_bin', 
           'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 
           'ps_ind_13_bin', 'id']

X_50 , y_50 = prepare_data(train_down_samp,to_drop)
y_50 = y_50.reshape([-1,1])

X_real_test, y_real_test = prepare_data(shuffle(train)[0:3000],to_drop)
y_real_test = y_real_test.reshape([-1,1])

X_train, X_test, y_train, y_test = train_test_split(X_50 , y_50, test_size=0.02)
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
N_FEATURES=X_train.shape[1]
num_classes = 1

173552   173552
3000   3000


In [53]:
tf.reset_default_graph()
input_size = X_train.shape[1]

# ### Placeholder variables
x = tf.placeholder(tf.float32, [None, N_FEATURES],name="X-input")
y_true = tf.placeholder(tf.float32, [None,num_classes],name="y-input") ##[None, num_classes]
is_train = tf.placeholder(tf.bool, name='is_train')
# ### Helper-functions for creating new variables
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name = 'weight')

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]),name='bias')


# ### Helper-function for creating a new Fully-Connected Layer
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True,  # Use Rectified Linear Unit (ReLU)?
                 use_sig=False):
    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)
    elif use_sig:
        layer = tf.nn.sigmoid(layer)
    return layer


# ###  Layers
#hyperparameters
n_nodes_hl1 = 37
n_nodes_hl2 = 35
n_nodes_hl3 = 15

beta = 0.002  # l2 regularization


keep_prob = tf.placeholder(tf.float32)
norm = tf.contrib.layers.batch_norm(x, is_training = is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope = 'bn')
layer_h1 = new_fc_layer(input=norm,
                         num_inputs=input_size,
                         num_outputs=n_nodes_hl1,
                         use_relu=True)
drop_h1 = tf.nn.dropout(layer_h1, 0.4 + keep_prob*0.6)

layer_h2 = new_fc_layer(input=drop_h1,
                         num_inputs=n_nodes_hl1,
                         num_outputs=n_nodes_hl2,
                         use_relu=False,
                         use_sig = True)

drop_h2 = tf.nn.dropout(layer_h2, 0.7 + keep_prob * 0.3)

layer_h3 = new_fc_layer(input=layer_h2,
                         num_inputs=n_nodes_hl2,
                         num_outputs=n_nodes_hl3,
                         use_relu=False,
                         use_sig = True)

drop_h3 = tf.nn.dropout(layer_h3, 0.5 + keep_prob * 0.5)

y_pred = new_fc_layer(input=drop_h1,
                         num_inputs=n_nodes_hl1,
                         num_outputs=num_classes,
                         use_relu=False,
                         use_sig = False)
y_pred_soft_max = tf.nn.sigmoid(y_pred)
#y_pred_cls = tf.argmax(y_pred, axis=1)




# ### Cost-function and Optimizer
learning_rate = tf.placeholder(tf.float32, shape=[])

#loss = tf.losses.mean_squared_error(y_true,y_pred_soft_max)
#loss = tf.losses.sigmoid_cross_entropy(y_true,y_pred) #,weights= y_true*10
loss = tf.square(y_pred_soft_max - y_true)/2

var_s = tf.trainable_variables()
loss_l2 = tf.add_n([tf.nn.l2_loss(v) for v in var_s if 'bias' not in v.name])*0.001

cost = tf.reduce_mean(loss)
#error = tf.square(y_pred - y_true)/2 
#cost = tf.reduce_sum(error + error* y_true*5)

optimizer = tf.train.AdamOptimizer().minimize(cost)  #  1e-4

# ### Performance measures
correct_prediction = tf.equal(tf.round(y_pred_soft_max), y_true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




session = tf.Session()
session.run(tf.global_variables_initializer())



In [54]:
from sklearn.metrics import precision_score
#feed_dict_test = {x: X_test, y_true: y_test, keep_prob: 1.0}  #.values.reshape([-1,1])
feed_dict_real_test = {x: X_real_test, y_true:y_real_test, keep_prob: 1.0,is_train:False}
####
from sklearn.metrics import roc_auc_score
def gini(y_actual, y_pred):
    return 2*roc_auc_score(y_actual, y_pred)-1
####
def print_accuracy( r_valu = False, feed = feed_dict_real_test):
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed)
    #print(precision_score(y_real_test,pred))
    if r_valu:
        return acc
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

print_accuracy()
from eval import normalized_gini
def print_gini():
    y_p,y = session.run([y_pred_soft_max,y_true], feed_dict=feed_dict_real_test)
    #print(y[:,1])
    return gini(y,y_p)
print(print_gini())
    
saver = tf.train.Saver()  #search tf.saver.tensorflow
best_validation_accuracy = 0
best_validation_gini = 0

Accuracy on test-set: 42.4%
0.11042450689


In [55]:
batch_size = 2048
def optimize( hm_epochs = 15, learning_r = 1e-4):
    global best_validation_accuracy,best_validation_gini
    global global_step
    for epoch in range(hm_epochs):
            epoch_loss = 0
            i = 0
            len_train = X_train.shape[0]
            while i < len_train:
                start = i
                end = i + batch_size

                batch_x = np.array(X_train[start:end])
                batch_y = np.array(y_train[start:end])    #.values.reshape([-1,1]))
                
                feed_dict_train = {x: batch_x,
                                   y_true: batch_y,
                                   keep_prob: 0.0,is_train:True}
                
                _, c = session.run([optimizer,cost], feed_dict=feed_dict_train)
                epoch_loss += c

                

                if i % 50000 == 0  or i == (len_train - 1) or i == 0:
                    

                    acc_ = print_accuracy(True)
                    if acc_ > best_validation_accuracy:
                        best_validation_accuracy = acc_
                        saver.save(session, 'models/my_model_acc')
                        imp_str_acc = '*'*4
                    else:
                        imp_str_acc = ''
                    gini = print_gini()
                    if gini > best_validation_gini:
                        best_validation_gini = gini
                        saver.save(session, 'models/my_model_gini')
                        imp_str_gini = '*'*4
                    else:
                        imp_str_gini = ''

                    print('epoch:{} loss: {:.4f},  acc: {:.4f}{} __gini: {:.4f}{}'.format(epoch,epoch_loss,acc_*100,imp_str_acc,gini,imp_str_gini))
                    epoch_loss = 0
                i += batch_size


    print('Epoch {} completed out of {}'.format(epoch+1,hm_epochs))

    print('Optimization ended '+ 10*'_' + 'best accuracy: {}'.format(best_validation_accuracy))

In [65]:
print(best_validation_gini)
print(print_gini())
print_accuracy()

0.255055571012
0.242839466401
Accuracy on test-set: 64.7%


In [66]:
optimize(600)

epoch:0 loss: 0.1178,  acc: 64.5667 __gini: 0.2429
epoch:1 loss: 0.1176,  acc: 64.7667 __gini: 0.2458
epoch:2 loss: 0.1156,  acc: 63.9000 __gini: 0.2437
epoch:3 loss: 0.1176,  acc: 64.3333 __gini: 0.2427
epoch:4 loss: 0.1159,  acc: 64.1000 __gini: 0.2417
epoch:5 loss: 0.1171,  acc: 63.9667 __gini: 0.2406
epoch:6 loss: 0.1178,  acc: 63.6000 __gini: 0.2400
epoch:7 loss: 0.1168,  acc: 63.7000 __gini: 0.2418
epoch:8 loss: 0.1173,  acc: 63.9000 __gini: 0.2455
epoch:9 loss: 0.1162,  acc: 63.9667 __gini: 0.2440
epoch:10 loss: 0.1171,  acc: 63.8333 __gini: 0.2433
epoch:11 loss: 0.1165,  acc: 64.1333 __gini: 0.2403
epoch:12 loss: 0.1166,  acc: 63.8000 __gini: 0.2404
epoch:13 loss: 0.1173,  acc: 63.8333 __gini: 0.2421
epoch:14 loss: 0.1173,  acc: 63.7667 __gini: 0.2413
epoch:15 loss: 0.1168,  acc: 63.7333 __gini: 0.2404
epoch:16 loss: 0.1163,  acc: 63.8667 __gini: 0.2379
epoch:17 loss: 0.1158,  acc: 64.3000 __gini: 0.2406
epoch:18 loss: 0.1166,  acc: 63.6667 __gini: 0.2427
epoch:19 loss: 0.1172,

KeyboardInterrupt: 

In [67]:
session.run(y_pred_soft_max,feed_dict_real_test)[:20]

array([[ 0.40503132],
       [ 0.37383202],
       [ 0.52046144],
       [ 0.495628  ],
       [ 0.36780107],
       [ 0.60976785],
       [ 0.31874985],
       [ 0.4654856 ],
       [ 0.23325017],
       [ 0.45791605],
       [ 0.43127698],
       [ 0.27411601],
       [ 0.56887966],
       [ 0.43301222],
       [ 0.53742951],
       [ 0.4060739 ],
       [ 0.42917451],
       [ 0.50185746],
       [ 0.3814179 ],
       [ 0.28370029]], dtype=float32)

In [68]:
test = pd.read_csv("data/test.csv")
print('test loaded')

#X_sub = transform_df(test)
X_sub = prepare_data(test,to_drop)
saver.restore(session,"models/my_model_acc")
sub1 = session.run(y_pred_soft_max,feed_dict={x:X_sub[:int(892816/2)],keep_prob:1.0,is_train:False})
sub2 = session.run(y_pred_soft_max,feed_dict={x:X_sub[int(892816/2):],keep_prob:1.0,is_train:False})
sub_tot_A = np.append(sub1,sub2)

test loaded
INFO:tensorflow:Restoring parameters from models/my_model_acc


In [69]:
saver.restore(session,"models/my_model_gini")
sub1 = session.run(y_pred_soft_max,feed_dict={x:X_sub[:int(892816/2)],keep_prob:1.0,is_train:False})
sub2 = session.run(y_pred_soft_max,feed_dict={x:X_sub[int(892816/2):],keep_prob:1.0,is_train:False})
sub_tot_B = np.append(sub1,sub2)


INFO:tensorflow:Restoring parameters from models/my_model_gini


In [71]:
test_id = test.id.values

sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = (sub_tot_B + sub_tot_B)/2

#series = pd.Series(sub_tot)
sub.to_csv('sub/submisssdsion_gini_{}.csv'.format(int(best_validation_accuracy*100)) ,index = False)